In [1]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
import os

paths = []

for root, dirs, files in os.walk("./DATA"):
    for file in files:
        if file.endswith(".nii"):
            paths.append(root + "/" + file)

In [7]:
paths

['../src/DATA//ABIDE_50054.nii']

In [9]:
batch_one_pixels = nib.load(paths[0]).get_fdata()

In [11]:
batch_one_pixels

memmap([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

In [40]:
pixels = [nib.load(x).get_fdata() for x in paths]
pixels[0:5]

[memmap([[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0.

In [79]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(2, 2), stride=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 1, kernel_size=(2, 2), stride=1, bias=False, padding=1),
            nn.Flatten()
        )
        
    def forward(self, x):
        return self.main(x)

In [80]:
model = Network()
print(model)

Network(
  (main): Sequential(
    (0): Conv2d(1, 32, kernel_size=(2, 2), stride=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 1, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
    (4): Flatten(start_dim=1, end_dim=-1)
  )
)


In [41]:
pixels = np.array([np.array(x) for x in pixels])

<ipython-input-41-fbfe15578550>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pixels = np.array([np.array(x) for x in pixels])


## Try Running a Model

On one slice of a single brain to see what the output may look like then
we want to try and extend this to an entire brain then try to expand this
to an n case.

In [59]:
brn_one = pixels[0]
single_slice = brn_one[0]
single_slice

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [69]:
x = single_slice[None, None, :, :]
x.shape

(1, 1, 228, 200)

In [82]:
out = model.forward(torch.Tensor(x))

In [86]:
alayer = nn.Linear(45600, 1)
alayer(out)

tensor([[-0.1366]], grad_fn=<AddmmBackward>)

In [91]:
brn_one.shape

(173, 228, 200)

## Extend this idea to an entire Brain

we want to take the idea of the single slice and proceed to do this for the entire brain

In [96]:
brn_one = brn_one[:, None, :, :]

In [97]:
brn_one.shape

(173, 1, 228, 200)

In [99]:
out = model.forward(torch.Tensor(brn_one))

In [100]:
out.shape

torch.Size([173, 45600])

In [104]:
psum = sum(alayer(out))

In [109]:
norm = nn.Tanh()
norm(psum)

tensor([1.], grad_fn=<TanhBackward>)